# GraphGuard

***Locate and find Classes in Apks with updated Obfuscation Mapping***


Processing Steps:
1. String Strategy (Classes + Methods) \
  Strategy using 2 different variants only involving Strings:
  * String Counter: used in Classes and Methods and try to find exact matching counter.
  * Unique Strings: Match Classes by identifying Strings used only in this single Class.



2. Structure Strategy (Classes) \
  Strategy that enumerates all classes and tries to find an optimal match by using the following (weighted) criteria:
  * Modifiers of class
  * Modifiers, Parameters, Parameter Types and Return Types of Methods
  * Number and Types of Fields.



3. Method Strategy (Methods) \
  Strategy that uses already matched classes and tries to find an optimal Method Match in the class by using the following (weighted) criteria:
  * Modifiers
  * Return Type, Parameter Types
  * Bytecode Length
  * References to and from


4. Field Strategy (Fields) \
  Strategy that uses already matched classes and tries to find an optimal Field Match in the class by using the following (weighted criteria)
  * By Type if a matching Type has been found
  * By weighted sub-criteria such as:
    * Access Flags
    * Size
    * Number of reads & Number of writes (references)

***If you use Akrolyb, please have a look at [Akrolyb Interoptability](#Interoptability-with-Akrolyb)***

In [1]:
%matplotlib notebook

%load_ext autoreload

from IPython.core.display import display, HTML
display(HTML("<style>div.output_area pre {white-space: pre;}</style>"))

In [2]:
%autoreload
from collections.abc import Iterable

from core.start import process_files
from utils.formats import *

from core.accumulator import Accumulator
from core.decs import *

from strategies import\
    strings as strings_strategy,\
    methods as methods_strategy,\
    structures as structures_strategy,\
    fields as fields_strategy

from core.strategy_handler import StrategyHandler, FLAG_CLASS, FLAG_METHOD, FLAG_FIELD

from utils import generate
from utils import io_akrolyb

# Loading Androguard

The following code loads the files and starts Androguard

It should support multiprocessing, however the Pipe communication seems to break when transmitting the processed Androguard Objects. I suspect the Object is simply too big for Pickle to serialize or another component in the transmitting chain.

In [3]:
AG_SESSION_FILE = "./Androguard.ag"
MULTIPROCESS_FILES = False  # Currently not working due to serialization issues


# APK Files to load
file_paths = (
#     "../../../Downloads/com.snapchat.android_10.87.5.69-2074_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk",
    "../../../Downloads/com.snapchat.android_10.88.0.63-2076_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk",
    "../../../Downloads/com.snapchat.android_10.88.5.72-2077_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk"
)

file_paths = (
#     "../../../Downloads/com.instagram.android_143.0.0.25.121-216817305_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk",
    
#     "../../../Downloads/com.instagram.android_151.0.0.23.120-232868015_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk",
#     "../../../Downloads/com.instagram.android_151.0.0.23.120-232867993_minAPI19(armeabi-v7a)(nodpi)_apkmirror.com.apk",
    
#     "../../../Downloads/com.instagram.android_152.0.0.25.117-234847234_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk",
#     "../../../Downloads/com.instagram.android_152.0.0.25.117-234847224_minAPI19(armeabi-v7a)(nodpi)_apkmirror.com.apk",
    
    "../../../Downloads/com.instagram.android_153.0.0.34.96-236572355_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk",
    "../../../Downloads/com.instagram.android_153.0.0.34.96-236572319_minAPI19(armeabi-v7a)(nodpi)_apkmirror.com.apk"
)

In [4]:
(a, d, dx), (a2, d2, dx2) = process_files(*file_paths, MULTIPROCESS_FILES)

Loading Session from Apk at ../../../Downloads/com.instagram.android_153.0.0.34.96-236572355_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk
Loading Session from Apk at ../../../Downloads/com.instagram.android_153.0.0.34.96-236572319_minAPI19(armeabi-v7a)(nodpi)_apkmirror.com.apk


# List of Classes, Methods and Variables

Defining Elements that GraphGuard should try to match (obviously requires full class names).
* `named_c_decs` in form of a tuple of Strings
* `named_m_decs` in form of a tuple of MethodDeclarations
* `named_f_decs` in form of a tuple of FieldDeclarations

In [5]:
# Classes of the Methods and Fields are automatically merged with c_decs
named_c_decs = {
    'MAIN_ACTIVITY': 'com.instagram.mainactivity.MainActivity',
    'CLIPITEM': 'X.6P4',
    'COMMENT_LONGCLICK_HANDLER': 'X.3Pr',
    'DEVELOPEROPTIONSLAUNCHER': 'com.instagram.debug.devoptions.api.DeveloperOptionsLauncher',
    'DIRECT_VISUAL_MESSAGE_VIEW_CONTROLLER': 'com.instagram.direct.visual.DirectVisualMessageViewerController',
    'DISCOVER_ADS': 'X.1Zk',
    'DM_GESTUREDETECTOR': 'X.3KI',
    'DM_OBJECT': 'X.2fu',
    'DM_HOLDER': 'X.56q',
    'DM_REPLY': 'X.41v',
    'DM_VOICEMESSAGE': 'X.57c',
    'FEEDITEM': 'X.1ZQ',
    'FRAGMENT': 'X.1Ju',
    'HTTPREQUESTTASKFACTORY': 'X.0nY',
    'INAPP_NOTIFICATION': 'X.65A',
    'IGTV_VIEWER_FRAGMENT': 'X.7Es',
    'REALTIME_CLIENT_MANAGER': 'com.instagram.realtimeclient.RealtimeClientManager',
    'JACKSON_OBJ': 'X.0zx',
    'JACKSON_CREATOR': 'X.0hs',
    'LIVE_POST': 'X.1yx',
    'MAINFEED': 'X.1KR',
    'MEDIAITEM': 'X.1RK',
    'MEDIAJSON_HELPER': 'com.instagram.feed.media.Media__JsonHelper',
    'NOTIFICATION': 'X.1vx',
    'NOTIFICATION_HANDLER': 'X.2fy',
    'PHOTOTIMERCONTROLLER': 'X.39u',
    'POSTDETAIL': 'X.1Xg',
    'POSTGESTURE_SIMPLEPOST': 'X.1ma',
    'POSTGESTURE_CAROUSELIMAGE': 'X.20w',
    'POSTGESTURE_CAROUSELVIDEO': 'X.2Ef',
    'POSTVIEW': 'X.1Xm',
    'PROFILE': 'X.0kG',
    'PROFILEDETAIL': 'X.2vc',
    'PROFILEPIC_LONGCLICK': 'X.2vk',
    'PROXYGEN_JNI_HANDLER': 'com.facebook.proxygen.JniHandler',
    'REELVIDEOPLAYER': 'X.29E',
    'REEL_VIEWER_FRAGMENT': 'com.instagram.reels.fragment.ReelViewerFragment',
    'NOTIFICATION_RUNNABLE': 'X.2ei',
    'SERIALSCHEDULER': 'X.0hj',
    'SESSION': 'X.0F2',
    'SIMPLE_EXECUTOR': 'X.4J0',
    'SWIPE_NAVIGATION_CONTAINER': 'com.instagram.ui.swipenavigation.SwipeNavigationContainer',
    'USER': 'X.1SK',
    'VIDEOSETTINGSMANAGER': 'X.1aL',
    'VOLUME_INDICATOR': 'com.instagram.ui.widget.volume.VolumeIndicator'
}

In [6]:
named_m_decs = {
    'ACTIVITY_ON_DESTROY': MethodDec('com.instagram.mainactivity.MainActivity', 'onDestroy'),
    'BITMAP_COMPRESS': MethodDec('android.graphics.Bitmap', 'compress', 'android.graphics.Bitmap$CompressFormat', 'int', 'java.io.OutputStream'),
    'CLIPITEM_CONSTRUCTOR': MethodDec('X.6P4', '<init>', 'X.20S', 'X.8pv', 'android.view.View', 'X.7Ea', 'X.5rH'),
    'COMMENT_ONLONGCLICK': MethodDec('X.3Pr', 'onLongPress', 'android.view.MotionEvent'),
    'DEVELOPEROPTIONSLAUNCHER_LOADANDLAUNCHDEVELOPEROPTIONS': MethodDec('com.instagram.debug.devoptions.api.DeveloperOptionsLauncher', 'loadAndLaunchDeveloperOptions', 'android.content.Context', 'X.1D7', 'androidx.fragment.app.FragmentActivity', 'X.0F2', 'android.os.Bundle'),
    'DEVELOPEROPTIONSLAUNCHER_LAUNCHSTORYTOOLBARSWITCHERTOOL': MethodDec('com.instagram.debug.devoptions.api.DeveloperOptionsLauncher', 'launchStoryToolbarSwitcherTool', 'android.content.Context', 'androidx.fragment.app.FragmentActivity', 'X.0F2'),
    'DEVELOPEROPTIONSLAUNCHER_LAUNCHSTORIESEXPERIMENTSWITCHERTOOL': MethodDec('com.instagram.debug.devoptions.api.DeveloperOptionsLauncher', 'launchStoriesExperimentSwitcherTool', 'android.content.Context', 'androidx.fragment.app.FragmentActivity', 'X.0F2'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_PAUSE': MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0D', 'com.instagram.direct.visual.DirectVisualMessageViewerController', 'java.lang.String'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_RESUME': MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0E', 'com.instagram.direct.visual.DirectVisualMessageViewerController', 'java.lang.String'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_SETSTATE': MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0C', skip_params=True),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_STARTVIEW': MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A05', 'com.instagram.direct.visual.DirectVisualMessageViewerController'),
    'DMGESTUREDETECTOR_ONLONGPRESS': MethodDec('X.3KI', 'onLongPress', 'android.view.MotionEvent'),
    'DMHOLDER_GETCURRENTDMMEDIA': MethodDec('X.56q', 'A01'),
    'DMREPLYCALLBACK_RUN': MethodDec('X.41v', 'A0E', 'X.41v', 'int'),
    'DMREPLY_SETSEEN': MethodDec('X.41v', 'A0B', 'X.41v'),
    'FEEDITEM_JSON_GENERATE': MethodDec('X.1ZQ', 'A00', 'X.0iA'),
    'FRAGMENT_GETACTIVITY': MethodDec('X.1Ju', 'getActivity'),
    'HTTPREQUESTTASKFACTORY_CREATEREQUEST': MethodDec('X.0nY', 'A03', 'java.lang.String', 'X.0F2'),
    'INAPPNOTIFICATION_RUN': MethodDec('X.65A', 'run'),
    'IGTVVIEWERFRAGMENT_GETMEDIA': MethodDec('X.7Es', 'A00', 'X.7Es'),
    'IGTVVIEWERFRAGMENT_UPDATESTATE': MethodDec('X.7Es', 'A0B', 'X.7Es'),
    'INDICATEACTIVITY_DO': MethodDec('com.instagram.realtimeclient.RealtimeClientManager', 'sendCommand', 'java.lang.String', 'java.lang.String', 'com.instagram.realtimeclient.RealtimeClientManager$MessageDeliveryCallback'),
    'JACKSON_CLOSE': MethodDec('X.0zx', 'close'),
    'JACKSON_CREATEGENERATOR': MethodDec('X.0hs', 'A04', 'java.io.OutputStream', 'java.lang.Integer'),
    'JACKSON_JSONFACTORY': MethodDec('X.0hs', '<init>', 'X.Cnl'),
    'LIVE_POST_JSON_GENERATE': MethodDec('X.1yx', 'A00', 'X.0iz', 'X.1yy'),
    'MAINACTIVITY_GETCHILDFRAGMENTMANAGER': MethodDec('com.instagram.mainactivity.MainActivity', 'AKg'),
    'MAINFEED_CONFIGUREACTIONBAR': MethodDec('X.1KR', 'configureActionBar', 'X.1GV'),
    'MEDIAITEM_JSON_GENERATE': MethodDec('X.1RK', 'A00', 'X.0iA', 'boolean'),
    'MEDIA_JSON_GENERATE': MethodDec('com.instagram.feed.media.Media__JsonHelper', 'A01', 'X.0iz', 'X.1RK'),
    'NOTIFICATIONINST_GETJSON': MethodDec('X.1vx', 'A01'),
    'NOTIFICATION_SHOW': MethodDec('X.2fy', 'A01', 'android.content.Context', 'java.lang.CharSequence', 'int'),
    'PHOTOTIMERCONTROLLER_INITPICTURECOUNTDOWN': MethodDec('X.39u', 'A01'),
    'POSTDETAIL_SETDATA': MethodDec('X.1Xg', 'A05', skip_params=True),
    'POSTGESTURE_SIMPLEPOST_ONDOUBLETAP': MethodDec('X.1ma', 'onDoubleTap', 'android.view.MotionEvent'),
    'POSTGESTURE_CAROUSELIMAGE_ONDOUBLETAP': MethodDec('X.20w', 'onDoubleTap', 'android.view.MotionEvent'),
    'POSTGESTURE_CAROUSELVIDEO_ONDOUBLETAP': MethodDec('X.2Ef', 'onDoubleTap', 'android.view.MotionEvent'),
    'POSTVIEW_ADDMEDIA': MethodDec('X.1Xm', 'A01', skip_params=True),
    'PROFILEDETAIL_SETDATA': MethodDec('X.2vc', 'A02', skip_params=True),
    'PROFILE_ISSELF': MethodDec('X.0kG', 'A05', 'X.0F2', 'X.0ik'),
    'PROFILE_ONLONGCLICK': MethodDec('X.2vk', 'A00', skip_params=True),
    'PROXYGEN_SENDREQUESTWITHBODY': MethodDec('com.facebook.proxygen.JniHandler', 'sendRequestWithBodyAndEom', 'org.apache.http.client.methods.HttpUriRequest', 'byte[]', 'int', 'int'),
    'REELVIDEOPLAYER_SEEKTO': MethodDec('X.29E', 'A0T', 'int'),
    'REELVIEWERFRAGMENT_ADVANCETONEXTREEL': MethodDec('com.instagram.reels.fragment.ReelViewerFragment', 'A1I', 'X.1nf'),
    'REELVIEWERFRAGMENT_PAUSEREEL': MethodDec('com.instagram.reels.fragment.ReelViewerFragment', 'A0o', 'com.instagram.reels.fragment.ReelViewerFragment', 'java.lang.String'),
    'REELVIEWERFRAGMENT_RESUMEREEL': MethodDec('com.instagram.reels.fragment.ReelViewerFragment', 'A0S', 'com.instagram.reels.fragment.ReelViewerFragment'),
    'REELVIEWERFRAGMENT_UPDATEMEDIA': MethodDec('com.instagram.reels.fragment.ReelViewerFragment', 'A0z', 'X.1gM', 'X.1nf'),
    'REGULARNOTIFICATION_RUN': MethodDec('X.2ei', 'run'),
    'SERIALSCHEDULER_EXECUTE': MethodDec('X.0hj', 'A02', 'X.0oT'),
    'SESSION_GETTOKEN': MethodDec('X.0F2', 'getToken'),
    'SIMPLEEXECUTOR_ONFAIL': MethodDec('X.4J0', 'onFail', 'X.228'),
    'SIMPLEEXECUTOR_ONSUCCESS': MethodDec('X.4J0', 'onSuccess', 'java.lang.Object'),
    'SIMPLEXECUTOR_CONSTRUCTOR': MethodDec('X.4J0', '<init>', 'X.0F2'),
    'SWIPENAVIGATIONCONTAINER_ONTOUCHEVENT': MethodDec('com.instagram.ui.swipenavigation.SwipeNavigationContainer', 'onTouchEvent', 'android.view.MotionEvent'),
    'USER_JSON_GENERATE': MethodDec('X.1SK', 'A03', 'X.0iz', 'X.0ij'),
    'VIDEOSETTINGSMANAGER_ISAUTOPLAYDISABLED': MethodDec('X.1aL', 'A00', 'X.0F2'),
    'VOLUMEINDICATOR_SETPROGRESS': MethodDec('com.instagram.ui.widget.volume.VolumeIndicator', 'A00', 'int', 'int')
}

In [7]:
named_f_decs = {
    'ACTIONBAR_BASEVIEW': FieldDec('X.1GU', 'A08'),
    'CLIPSHOLDER_MEDIA': FieldDec('X.20S', 'A00'),
    'COMMENTTOUCHHANDLER_COMMENT': FieldDec('X.3Pr', 'A04'),
    'COMMENT_TEXT': FieldDec('X.1UD', 'A0X'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_DMHOLDER': FieldDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0D'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_ROOTVIEW': FieldDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'mRootView'),
    'DMGESTUREDETECTOR_MESSAGEOBJECT': FieldDec('X.3KI', 'A01'),
    'DMHOLDER_MEDIA': FieldDec('X.96J', 'A07'),
    'DMVOICEMESSAGE_RAWMEDIA': FieldDec('X.57c', 'A04'),
    'FEEDITEM_TYPE': FieldDec('X.1ZQ', 'A0I'),
    'HTTPREQUESTTASK_EXECUTOR': FieldDec('X.0oS', 'A00'),
    'IGTVVIEWERFRAGMENT_ROOTVIEW': FieldDec('X.7Es', 'A0E'),
    'MAINFEED_SESSION': FieldDec('X.1KR', 'A0S'),
    'MEDIANOTIFICATION_LIST': FieldDec('X.1Zb', 'A06'),
    'MEDIA_WRAPPER_LIVEMEDIAVAR': FieldDec('X.1nf', 'A09'),
    'MEDIA_WRAPPER_REELMEDIAVAR': FieldDec('X.1nf', 'A08'),
    'NOTIFICATION_INSTANCE': (FieldDec('X.65A', 'A00'), FieldDec('X.2ei', 'A00')),
    'NOTIFICATION_SESSION': (FieldDec('X.65A', 'A01'), FieldDec('X.2ei', 'A01')),
    'POSTDETAILUI_PROFILENAME': FieldDec('X.1kQ', 'A05'),
    'POST_VIEW_COMMENTBUTTON': FieldDec('X.1kT', 'A02'),
    'PROFILEDETAILUI_BIOTEXT': FieldDec('X.2vg', 'A0N'),
    'PROFILEDETAILUI_FOLLOWER': FieldDec('X.2vg', 'A00'),
    'PROFILEDETAILUI_FOLLOWING': FieldDec('X.2vg', 'A01'),
    'PROFILEDETAILUI_FULLNAME': FieldDec('X.2vg', 'A0A'),
    'PROFILE_PROFILEPIC': FieldDec('X.2vi', 'A09'),
    'REELVIDEOPLAYER_CONTROLLER': FieldDec('X.3A0', 'A0D'),
    'REEL_VIEWER_FRAGMENT_SESSION': FieldDec('com.instagram.reels.fragment.ReelViewerFragment', 'A1K'),
    'REEL_VIEWER_FRAGMENT_VIDEOPLAYER': FieldDec('com.instagram.reels.fragment.ReelViewerFragment', 'mVideoPlayer'),
    'REEL_VIEWER_FRAGMENT_VIEWROOT': FieldDec('com.instagram.reels.fragment.ReelViewerFragment', 'mViewRoot'),
    'STORYUSER_SESSION': FieldDec('X.1nf', 'A0D')
}

In [8]:
m_decs = tuple(named_m_decs.values())
f_decs = []
for f_dec in named_f_decs.values():
    if isinstance(f_dec, Iterable):
        f_decs.extend(f_dec)
    else:
        f_decs.append(f_dec)
f_decs = tuple(f_decs)

# Add Classes of Methods and Fields to c_decs
c_decs = tuple(map(lambda m: m.class_name, m_decs)) \
        + tuple(map(lambda v: v.class_name, f_decs)) + tuple(named_c_decs.values())

# Processing and Matching

Loading the accumulator, an object that manages all the possible candidates that are matched by the different Matchers, and extracts the matching candidates. It also performs Inner joins on previous candidates to find the exact (or optimal) match.

In [9]:
accumulator = Accumulator()

Resolving the previously defined MethodDecs. If this fails, the MethodDecs are wrong and contain an error. Make sure the method specified with the MethodDec exists.

In [10]:
dec_cas = resolve_classes(dx, c_decs)

r_cas = tuple(dec_cas.values())
r_mas = resolve_methods(m_decs, dec_cas)
r_fas = resolve_fields(f_decs, dec_cas)

print("Resolved all Classes, Methods and Fields")
if False:
    print("", *map(pretty_format_ma, r_mas), sep="\n* ")

# Arguments to provide to Strategies
s_args = (dx, dx2, r_cas, r_mas, r_fas, accumulator)

Resolved all Classes, Methods and Fields


## Strategies & StrategyHandler

Strategies are essentially functions returning candidates for Classes, Methods or Fields. A `strategy` gets registered in the `StrategyHandler` with Flags indicating whether it tries to match Classes, Methods or Fields. If a search is invoked for a matching flag, the strategy gets invoked on the given arguments.

Global Parameters and Rules of the Strategies allow for easily changing criteria, e. g. use a more lenient search and not require strict matches.

In [11]:
strategies = StrategyHandler()


strings_strategy.MAX_USAGE_COUNT_STR = 20
strings_strategy.UNIQUE_STRINGS_MAJORITY = 2 / 3

methods_strategy.MIN_MATCH_POINTS = 2

## String Strategy

### Exact Counter Match

Extracts Strings used either in the given methods directly or in the classes the methods define for both, the old version and the new version. It then compares the Counters for classes and methods and tries to find exact matches between the Counter Objects.

In [12]:
def string_counter_strategy(r_cas, r_mas, r_fas):
    string_s = strings_strategy.StringStrategy(dx, dx2, r_cas, r_mas, r_fas, accumulator)
    candidates_cs, candidates_ms = string_s.compare_counters()
    
    accumulator.add_candidates(candidates_cs, candidates_ms)

strategies.add_strategy(string_counter_strategy, FLAG_CLASS | FLAG_METHOD)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

+ Found single candidate for Method. Considering it a match 
	com.instagram.reels.fragment.ReelViewerFragment#A0z(X.1gM, X.1nf) -> com.instagram.reels.fragment.ReelViewerFragment#A0y(X.1gc, X.1nw)
+ Found single candidate for Method. Considering it a match 
	X.1vx#A01() -> X.1wE#A01()
+ Found single candidate for Method. Considering it a match 
	X.2vc#A02(X.2vg, android.content.Context, X.0F2, X.0ij, X.0Ru, com.instagram.profile.fragment.UserDetailDelegate, boolean, X.1RK, java.lang.Integer, boolean, boolean, X.1bM) -> X.2w1#A02(X.2w5, android.content.Context, X.0F2, X.0iu, X.0S6, com.instagram.profile.fragment.UserDetailDelegate, boolean, X.1RY, java.lang.Integer, boolean, boolean, X.1ba)
+ Found single candidate for Method. Considering it a match 
	com.instagram.realtimeclient.RealtimeClientManager#sendCommand(java.lang.String, java.lang.String, com.instagram.realtimeclient.RealtimeClientManager$MessageDeliveryCallback) -> com.instagram.realtimeclient.RealtimeClientManager#sendComman

### Unique Strings

Gather all Strings that are used only in a single class ("Unique Strings") that we still need to match. Then try to find the matching class by only searching for the Unique Strings.

In [13]:
def unique_strings_strategy(r_cas, r_mas, r_fas):
    string_s = strings_strategy.StringStrategy(dx, dx2, r_cas, r_mas, r_fas, accumulator)
    candidates_cs = string_s.compare_unique_strings()
    
    accumulator.add_candidates(candidates_cs)

strategies.add_strategy(unique_strings_strategy, FLAG_CLASS)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

+ Found single candidate for Class. Considering it a match! 
	LX/0F2; -> LX/0F2;
+ Found single candidate for Class. Considering it a match! 
	LX/0hj; -> LX/0hu;
+ Found single candidate for Class. Considering it a match! 
	LX/0nY; -> LX/0nj;
+ Found single candidate for Class. Considering it a match! 
	LX/0oS; -> LX/0od;
+ Found single candidate for Class. Considering it a match! 
	LX/0zx; -> LX/108;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/mainactivity/MainActivity; -> Lcom/instagram/mainactivity/MainActivity;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/ui/swipenavigation/SwipeNavigationContainer; -> Lcom/instagram/ui/swipenavigation/SwipeNavigationContainer;
+ Found single candidate for Class. Considering it a match! 
	LX/1GU; -> LX/1Gi;
+ Found single candidate for Class. Considering it a match! 
	LX/1Ju; -> LX/1K8;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/feed/media/Media__Json

## Structure Strategy

Iterating through every single class and checks for each unmatched class if both have a similar "Profile":
* Number of Methods and Fields
* Types of Fields and Descriptors of Methods

In [14]:
def structure_strategy(r_cas, r_mas, r_fas):
    structure_s = structures_strategy.StructureStrategy(dx, dx2, r_cas, r_mas, r_fas, accumulator)
    candidates_cs = structure_s.get_exact_structure_matches()

    accumulator.add_candidates(candidates_cs)

strategies.add_strategy(structure_strategy, FLAG_CLASS)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

+ Found single candidate for Class. Considering it a match! 
	Landroid/graphics/Bitmap; -> Landroid/graphics/Bitmap;
+ Found single candidate for Class. Considering it a match! 
	LX/6P4; -> LX/7Bu;
+ Found single candidate for Class. Considering it a match! 
	LX/3Pr; -> LX/3QK;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/debug/devoptions/api/DeveloperOptionsLauncher; -> Lcom/instagram/debug/devoptions/api/DeveloperOptionsLauncher;
+ Found single candidate for Class. Considering it a match! 
	LX/3KI; -> LX/3Kl;
+ Found single candidate for Class. Considering it a match! 
	LX/56q; -> LX/4yp;
+ Found single candidate for Class. Considering it a match! 
	LX/0hs; -> LX/0i3;
+ Found single candidate for Class. Considering it a match! 
	LX/1ma; -> LX/1mr;
* Found multiple candidates for Class LX/20w;
* Found multiple candidates for Class LX/2Ef;
+ Found single candidate for Class. Considering it a match! 
	LX/1Xm; -> LX/1Y0;
+ Found single candidate for Class.

## Method Strategy

Uses different weighted criteria to get exact or optimal matches. The criteria are:
* Modifiers
* Return Type and Parameter Types
* Length of Bytecode
* References to and from

In [15]:
def method_strategy(r_cas, r_mas, r_fas):
    method_s = methods_strategy.MethodStrategy(dx, dx2, r_cas, r_mas, r_fas, accumulator)

    print("Exact Matching")
    print()

    # Exact Matches
    candidates_ms = method_s.try_resolve_ms(True)
    accumulator.add_candidates(candidates_ms=candidates_ms)

    print()
    print("Non-Exact Matching")
    print()

    method_s.update_matched()
    # Non-Exact Matches by using weights on the criteria
    candidates_ms = method_s.try_resolve_ms(False)
    accumulator.add_candidates(candidates_ms=candidates_ms)
    
strategies.add_strategy(method_strategy, FLAG_METHOD)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

Exact Matching

Class not resolved for Method X.39u#A01()
Class not resolved for Method X.20w#onDoubleTap(android.view.MotionEvent)
Class not resolved for Method X.2Ef#onDoubleTap(android.view.MotionEvent)
+ Found single candidate for Method. Considering it a match 
	com.instagram.mainactivity.MainActivity#onDestroy() -> com.instagram.mainactivity.MainActivity#onDestroy()
+ Found single candidate for Method. Considering it a match 
	android.graphics.Bitmap#compress(android.graphics.Bitmap$CompressFormat, int, java.io.OutputStream) -> android.graphics.Bitmap#compress(android.graphics.Bitmap$CompressFormat, int, java.io.OutputStream)
+ Found single candidate for Method. Considering it a match 
	X.6P4#<init>(X.20S, X.8pv, android.view.View, X.7Ea, X.5rH) -> X.7Bu#<init>(X.20j, X.7CY, android.view.View, X.7CJ, X.7CV)
+ Found single candidate for Method. Considering it a match 
	X.3Pr#onLongPress(android.view.MotionEvent) -> X.3QK#onLongPress(android.view.MotionEvent)
+ Found single candida

## Field Strategy

Gathers all fields of matching Class and filters using the following criteria:
* By Type if a matching Type has been found
* By weighted sub-criteria such as:
  * Access Flags
  * Size
  * Number of reads & Number of writes
* By Index if the list of Fields has not changed dramatically

In [16]:
def field_strategy(r_cas, r_mas, r_fas):
    field_s = fields_strategy.FieldStrategy(*s_args)
    
    print("Resolving Types of Fields")
    strategies.invoke_strategies(tuple(field_s.get_types_to_match()))
    print()
    
    candidates_fs = field_s.try_resolve_fs()
    
    accumulator.add_candidates(candidates_fs=candidates_fs)
    
strategies.add_strategy(field_strategy, FLAG_FIELD)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

Resolving Types of Fields
Could resolve 0 new Classes, 0 new Methods, 0 new Fields.
+ Found single candidate for Class. Considering it a match! 
	LX/0ij; -> LX/0iu;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/common/ui/widget/imageview/IgImageView; -> Lcom/instagram/common/ui/widget/imageview/IgImageView;
+ Found single candidate for Class. Considering it a match! 
	LX/1ZR; -> LX/1Zf;
+ Found single candidate for Class. Considering it a match! 
	LX/1yy; -> LX/1zF;
Could resolve 4 new Classes, 0 new Methods, 0 new Fields.
+ Found single candidate for Class. Considering it a match! 
	LX/0oW; -> LX/0oh;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/igtv/ui/GestureManagerFrameLayout; -> Lcom/instagram/igtv/ui/GestureManagerFrameLayout;
+ Found single candidate for Class. Considering it a match! 
	LX/0QZ; -> LX/0Qi;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/common/ui/colorfilter/ColorFilterAlp

# Results
Display summary and matching pairs.

Output to compatible MethodDec input to update from auto-updated values.

In [17]:
print(len(accumulator.matching_cs), "/", len(named_c_decs), "Classes were matched")
print(len(accumulator.matching_ms), "/", len(m_decs), "Methods were matched")
print(len(accumulator.matching_fs), "/", len(f_decs), "Fields were matched")

print()
print("Matching Classes:")
for c1, c2 in accumulator.matching_cs.items():
    print("•", pretty_format_class(c1), "->", pretty_format_class(c2))

print()
print("Matching Methods:")
for m, ma in accumulator.matching_ms.items():
    print("•", pretty_format_ma(m), "->", pretty_format_ma(ma))

print()
print("Matching Fields:")
for fa, fa2 in accumulator.matching_fs.items():
    print("•", pretty_format_fa(fa), "->", pretty_format_fa(fa2))

65 / 45 Classes were matched
52 / 57 Methods were matched
30 / 32 Fields were matched

Matching Classes:
• com.instagram.reels.fragment.ReelViewerFragment -> com.instagram.reels.fragment.ReelViewerFragment
• X.1vx -> X.1wE
• X.2vc -> X.2w1
• com.instagram.realtimeclient.RealtimeClientManager -> com.instagram.realtimeclient.RealtimeClientManager
• X.41v -> X.42P
• com.instagram.direct.visual.DirectVisualMessageViewerController -> com.instagram.direct.visual.DirectVisualMessageViewerController
• X.1KR -> X.1Kf
• X.1RK -> X.1RY
• X.1aL -> X.1aZ
• X.29E -> X.29V
• X.2ei -> X.2f7
• X.4J0 -> X.4Io
• X.65A -> X.61P
• X.0F2 -> X.0F2
• X.0hj -> X.0hu
• X.0nY -> X.0nj
• X.0oS -> X.0od
• X.0zx -> X.108
• com.instagram.mainactivity.MainActivity -> com.instagram.mainactivity.MainActivity
• com.instagram.ui.swipenavigation.SwipeNavigationContainer -> com.instagram.ui.swipenavigation.SwipeNavigationContainer
• X.1GU -> X.1Gi
• X.1Ju -> X.1K8
• com.instagram.feed.media.Media__JsonHelper -> com.instagr

• com.instagram.ui.swipenavigation.SwipeNavigationContainer#onTouchEvent(android.view.MotionEvent) -> com.instagram.ui.swipenavigation.SwipeNavigationContainer#onTouchEvent(android.view.MotionEvent)
• com.instagram.ui.widget.volume.VolumeIndicator#A00(int, int) -> com.instagram.ui.widget.volume.VolumeIndicator#A00(int, int)
• X.3KI#onLongPress(android.view.MotionEvent) -> X.3Kl#onLongPress(android.view.MotionEvent)
• X.1ZQ#A00(X.0iA) -> X.1Ze#A00(X.0iL)
• X.1Ju#getActivity() -> X.1K8#getActivity()
• X.1yx#A00(X.0iz, X.1yy) -> X.1zE#A00(X.0jA, X.1zF)
• com.instagram.feed.media.Media__JsonHelper#A01(X.0iz, X.1RK) -> com.instagram.feed.media.Media__JsonHelper#A01(X.0jA, X.1RY)
• X.0F2#getToken() -> X.0F2#getToken()
• X.1SK#A03(X.0iz, X.0ij) -> X.1SY#A03(X.0jA, X.0iu)

Matching Fields:
• LX/1GU;#A08 -> LX/1Gi;#A08
• LX/20S;#A00 -> LX/20j;#A00
• LX/3Pr;#A04 -> LX/3QK;#A04
• LX/1UD;#A0X -> LX/1UR;#A0X
• Lcom/instagram/direct/visual/DirectVisualMessageViewerController;#A0D -> Lcom/instagram/d

In [18]:
generate.generate_m_decs(m_decs, r_mas, accumulator.matching_ms)
generate.generate_c_decs(named_c_decs.values(), r_cas, accumulator.matching_cs)
generate.generate_f_decs(f_decs, r_fas, accumulator.matching_fs)

m_decs = (
    MethodDec('com.instagram.mainactivity.MainActivity', 'onDestroy'),
    MethodDec('android.graphics.Bitmap', 'compress', 'android.graphics.Bitmap$CompressFormat', 'int', 'java.io.OutputStream'),
    MethodDec('X.7Bu', '<init>', 'X.20j', 'X.7CY', 'android.view.View', 'X.7CJ', 'X.7CV'),
    MethodDec('X.3QK', 'onLongPress', 'android.view.MotionEvent'),
    MethodDec('com.instagram.debug.devoptions.api.DeveloperOptionsLauncher', 'loadAndLaunchDeveloperOptions', 'android.content.Context', 'X.1DI', 'androidx.fragment.app.FragmentActivity', 'X.0F2', 'android.os.Bundle'),
    # No match for Method com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#launchStoryToolbarSwitcherTool(android.content.Context, androidx.fragment.app.FragmentActivity, X.0F2),
    # No match for Method com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#launchStoriesExperimentSwitcherTool(android.content.Context, androidx.fragment.app.FragmentActivity, X.0F2),
    MethodDec('com.instagram.

In [19]:
c_file = "/home/jaqxues/CodeProjects/IdeaProjects/Instaprefs/packimpl/src/main/java/com/marz/instaprefs/packimpl/decs/armv8/ClassDecsV8.kt"
m_file = "/home/jaqxues/CodeProjects/IdeaProjects/Instaprefs/packimpl/src/main/java/com/marz/instaprefs/packimpl/decs/armv8/MemberDecsV8.kt"
f_file = "/home/jaqxues/CodeProjects/IdeaProjects/Instaprefs/packimpl/src/main/java/com/marz/instaprefs/packimpl/decs/armv8/VariableDecsV8.kt"

In [20]:
c_txt = io_akrolyb.replace_cs(c_file, accumulator)
m_txt = io_akrolyb.replace_ms(m_file, accumulator, named_m_decs)
f_txt = io_akrolyb.replace_fs(f_file, accumulator, named_f_decs)

No matching Method found for com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#launchStoryToolbarSwitcherTool(android.content.Context, androidx.fragment.app.FragmentActivity, X.0F2)
No matching Method found for com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#launchStoriesExperimentSwitcherTool(android.content.Context, androidx.fragment.app.FragmentActivity, X.0F2)
MethodDec HEADERGENERATOR_GENERATEFROMDEVICEINFO not registered
No matching Method found for X.39u#A01()
No matching Method found for X.20w#onDoubleTap(android.view.MotionEvent)
No matching Method found for X.2Ef#onDoubleTap(android.view.MotionEvent)
No matching Field found for X.1kQ#A05
No matching Field found for X.1kT#A02


In [21]:
print(c_txt)
print(m_txt)
print(f_txt)

package com.marz.instaprefs.packimpl.decs.armv8

import com.jaqxues.akrolyb.genhook.decs.ClassDec
import com.marz.instaprefs.packimpl.decs.AClassDecs

object ClassDecsV8: AClassDecs() {
    override val MAIN_ACTIVITY = ClassDec("com.instagram.mainactivity.MainActivity")

    override val CLIPITEM = ClassDec("X.7Bu")

    override val COMMENT_LONGCLICK_HANDLER = ClassDec("X.3QK")

    override val DEVELOPEROPTIONSLAUNCHER = ClassDec("com.instagram.debug.devoptions.api.DeveloperOptionsLauncher")

    override val DIRECT_VISUAL_MESSAGE_VIEW_CONTROLLER = ClassDec("com.instagram.direct.visual.DirectVisualMessageViewerController")

    override val DISCOVER_ADS = ClassDec("X.1Zy")

    override val DM_GESTUREDETECTOR = ClassDec("X.3Kl")

    override val DM_OBJECT = ClassDec("X.2gJ")

    override val DM_HOLDER = ClassDec("X.4yp")

    override val DM_REPLY = ClassDec("X.42P")

    override val DM_VOICEMESSAGE = ClassDec("X.5Ge")

    override val EXPLORE_ADS = /* TODO */ ClassDec("X.6WS")



}
package com.marz.instaprefs.packimpl.decs.armv8

import android.view.ViewGroup
import android.widget.FrameLayout
import android.widget.ImageView
import android.widget.LinearLayout
import android.widget.TextView
import com.jaqxues.akrolyb.genhook.decs.FieldClass
import com.jaqxues.akrolyb.genhook.decs.VariableDec
import com.marz.instaprefs.packimpl.decs.AVariableDecs

object VariableDecsV8: AVariableDecs() {
    @FieldClass(["X.1Gi"])
    override val ACTIONBAR_BASEVIEW = VariableDec<ViewGroup>("A08")

    @FieldClass(["X.20j"])
    override val CLIPSHOLDER_MEDIA = VariableDec<Any>("A00")

    @FieldClass(["X.3QK"])
    override val COMMENTTOUCHHANDLER_COMMENT = VariableDec<Any?>("A04")

    @FieldClass(["X.1UR"])
    override val COMMENT_TEXT = VariableDec<String>("A0X")

    @FieldClass(["com.instagram.direct.visual.DirectVisualMessageViewerController"])
    override val DIRECTVISUALMESSAGEVIEWCONTROLLER_DMHOLDER = VariableDec<Any>("A0D")

    @FieldClass(["com.instagram.direct.visu

# Interoptability with Akrolyb

The following Code snippet allows to "extract" MethodDecs from MemberDeclarations in Akrolyb. This automates the process of providing GraphGuard the hooks it should find. Just paste the output (tuple of MethodDecs) in the corresponding section in this Notebook.

## Extract

It is not in Python, since it would require a Kotlin Parser and evaluating imports, variables etc. Just executing the Constructors in Kotlin, then getting the values is much easier than static analysis. 

The `main` function can (and should) be run statically (locally on the machine and not on your Android) to build the list of `MethodDec`s that GraphGuard is supposed to match in an updated build of the target application. Adjust the MemberDeclarations Class to the Class where you declared the `MemberDeclaration`s.


```kotlin
fun main() {
    GraphGuard.printGeneratedDecs(ClassDecs, MemberDecs, VariableDecs)
}
```